In [35]:
from pymongo import MongoClient
import pandas as pd
from collections import defaultdict
import numpy as np

In [36]:
client = MongoClient(port=27018)
candidates = client.lbj.candidates

In [37]:
pipeline = [
            #{"$match": {"industry": {"$exists": True}}},
            {"$unwind": {
                "path": "$experiences",
                "includeArrayIndex": "job_index"
                }
            },
            {"$project": {
                "industry": 1,
                "job_index": 1,
                "place": "$experiences.place",
                "function": "$experiences.function",
                "duration": "$experiences.duration",
                "mission": "$experiences.missions"
                }}
            ]

In [38]:
agg = candidates.aggregate(pipeline)

In [39]:
def _get_element(doc, field_name):

    if not field_name in doc:
        return np.nan
    else:
        return doc[field_name]

def make_dataframe(curr):
    data = defaultdict(list)

    for doc in curr:
        data['_id'].append(doc['_id'])
        data['industry'].append(_get_element(doc,'industry'))
        data['job_index'].append(doc['job_index'])
        data['place'].append(_get_element(doc, 'place'))
        data['function'].append(_get_element(doc, 'function'))
        data['duration'].append(_get_element(doc, 'duration'))
        data['mission'].append(_get_element(doc, 'mission'))

    return pd.DataFrame(data)

In [40]:
df = make_dataframe(agg)

In [ ]:
df.to_pickle("/data/rali7/Tmp/solimanz/data/pickles/2017_11_28.pkl")

In [ ]:
print("hello")